# MIN MAX PREDICTIVE METHOD

In this notebook, we will implement the min max predictive method. Since we aim to a lossless compresion, so we need to be able to decode the compressed images. Here we propose an another method than the original method is the paper but we borrow the main ideas.

We suppose that we have a set of `N` very similar images and let `I_min` and `I_max` be the minimum image and the maximum image. \
The main idea is that, instead of storing the original pixel values, we will make a predictive scheme and store the difference between the predicted pixel values and the original pixel values.

For each pixel $p$ of image $I$, we denote $L_p = \lfloor m.\frac{I_p - I_\text{min.p}}{I_\text{max.p} - I_\text{min.p}} \rfloor$ the level of that pixel, and where $m$ is the number of possible levels that we can consider as a hyper parameter of our method. 

Hence, the predicted value is computed as $\hat{I_p} = I_\text{min.p} + \frac{L_p}{m}(I_\text{max.p} - I_\text{min.p})$, then we store the distance $I_p - \hat{I_p}$.




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as skm
from Outils.dataloader import load_CIFAR10

In [2]:
# Load the raw CIFAR-10 data.
cifar10_dir = 'Dataset/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
def image_encoder(I, Imin, Imax, m = 20, block_size = (2,2,1)):
    '''
    Image encoder of min max predictive method
    Input:  I - image of shape (H, W, 3)

    Return: encoded_image : numpy array of the same shape as channel
            level_image : numpy array of shape image.shape / block_size (by each dim)
    '''
    encoded_image = np.zeros(I.shape)
    level_image = m*(I - Imin)/(Imax - Imin)
    level_image = level_image.astype(int)
    level_image = skm.block_reduce(level_image, block_size = block_size, func = np.min)
    I_hat = Imin + np.floor(np.kron(level_image, np.ones(block_size))*(Imax - Imin)/m)
    encoded_image = I - I_hat

    return encoded_image, level_image

In [12]:
def image_decoder(I_encoded, level, Imin, Imax, m = 20, block_size = (2,2,1)):
    I_hat = Imin + np.floor(np.kron(level, np.ones(block_size))*(Imax - Imin)/m)
    return I_encoded + I_hat


In [94]:
Imin = np.min(X_train[:10], axis=0)
Imax = np.max(X_train[:10], axis=0)

In [101]:
I = X_train[0]
I_encoded, I_level = image_encoder(I, Imin, Imax)
I_decoded = image_decoder(I_encoded, I_level, Imin, Imax)

In [102]:
np.sum(I != I_decoded)

0

In [13]:
def MMPredictive_Encoder(X, m = 20, block_size = (2,2,1)):
    '''
    Min Max Predictive Encoder
    Input: X - numpy array of shape (N, H, W, C)
       block_size - block size used in predictive level
       m - number of levels 
    Return: Y - numpy array of shape (N, H, W, C), difference between predictive images and original images
    L - numpy array of shape (N, H/block_size, W/block_size, C) containing the levels  
    Imin
    Imax
    '''
    N = X.shape[0]
    Y = []
    L = []
    Imin = np.min(X, axis=0)
    Imax = np.max(X, axis=0)

    for i in range(N):
        Yi, Li = image_encoder(X[i], Imin, Imax, m = m, block_size=block_size)
        Y.append(Yi)
        L.append(Li)

    return np.array(Y), np.array(L), Imin, Imax

In [14]:
def MMPredictive_Decoder(Y, L, Imin, Imax, m = 20, block_size = (2,2,1)):
    '''
    Min Max Predictive Encoder
    Input: Y - numpy array of shape (N, H, W, C), difference between predictive images and original images
           L - numpy array of shape (N, H/block_size, W/block_size, C) containing the levels
           block_size - block size used in predictive level
           m - number of levels  
    Return:
    X - numpy array of shape (N, H, W, C)
    '''
    N = Y.shape[0]
    X = []

    for i in range(N):
        Xi = image_decoder(Y[i], L[i], Imin, Imax, m = m, block_size=block_size)
        X.append(Xi)

    return np.array(X)

Now we will check that our decoded images are identical to the original images

In [8]:
X_test[1].shape

(32, 32, 3)

In [15]:
X_encoded, L, Imin, Imax = MMPredictive_Encoder(X_test)
X_decoded = MMPredictive_Decoder(X_encoded, L, Imin, Imax)
#Hải lì ăn cục cớt

In [16]:
np.sum(X_decoded != X_test)

0